<a href="https://colab.research.google.com/github/rithikafjohnson/Autonomous-Farming-System-Crop-Analysis/blob/main/threshold_segmentation_%2B_som.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import os
import numpy as np
from skimage import io
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import img_as_ubyte, img_as_float
from scipy import ndimage 
from matplotlib import pyplot as plt
from matplotlib.colors import hsv_to_rgb
import cv2
from PIL import Image
from skimage.color import rgb2gray
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV 
import pickle
import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage.exposure import rescale_intensity

In [ ]:
!pip install minisom

In [ ]:
!unzip RiceDiseases-DataSet-master.zip

Archive:  RiceDiseases-DataSet-master.zip
   creating: RiceDiseases-DataSet-master/
   creating: RiceDiseases-DataSet-master/Bacterial leaf blight/
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_001.jpg  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_002.png  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_003.jpg  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_004.PNG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_005.jpg  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_006.PNG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_007.JPG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_008.PNG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_009.JPG  
  inflating: RiceDiseases-DataSet-master/Bacterial leaf blight/blight_orig_010.PNG  
  

In [ ]:
label_list=[]
image_list=[]
path1= 'RiceDiseases-DataSet-master'

listing=os.listdir(path1)
for file in listing:
    
    list2=os.listdir(path1+'/'+file)
    for image in list2:
        im=cv2.imread(path1+'/'+file+'/'+image)
        
        
        
        res_img = segmentation(im)
        

        
        image_list.append(res_img)
        label_list.append(file)



In [ ]:
def segmentation(im, mode=0):
    
    gray = None
    img_hsv = np.zeros((256,256,3))
    if mode==0:
        gray = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)
    

    #gray = rgb2gray(im)
    
    #Thresholding
    elif mode==1:
        img_hsv = cv2.cvtColor(np.float32(im),cv2.COLOR_BGR2HSV)
        gray = rgb2gray(img_hsv)
    

    
    
        gray_r = gray.reshape(gray.shape[0]*gray.shape[1])
        for i in range(gray_r.shape[0]):
            if gray_r[i] > gray_r.mean():
                gray_r[i] = 1
            else:
                gray_r[i] = 0
        gray = gray_r.reshape(gray.shape[0],gray.shape[1])

        gray = rgb2gray(im)
        gray_r = gray.reshape(gray.shape[0]*gray.shape[1])
        for i in range(gray_r.shape[0]):
            if gray_r[i] > 0.8 :
                gray_r[i] = 255
            elif gray_r[i] > 0.6:
                gray_r[i] = 128

            else:
                gray_r[i] = 0
        gray = gray_r.reshape(gray.shape[0],gray.shape[1])
    
    
    #Felzenszwalb Segmentation
    elif mode==2:
    
        image_felzenszwalb = seg.felzenszwalb(img_hsv)
        gray = cv2.cvtColor(image_felzenszwalb,cv2.COLOR_RGB2GRAY)
    
    # SLIC Segmentation
    elif mode==3:
    
        vis = np.zeros(im.shape[:2], dtype="float")

        segments = slic(img_as_float(im), n_segments=200,slic_zero=True)
        for v in np.unique(segments):
            mask = np.ones(im.shape[:2])
            mask[segments == v] = 0
            C = segment_colorfulness(im, mask)
            vis[segments == v] = C
        vis = rescale_intensity(vis, out_range=(0, 255)).astype("uint8")
        alpha = 0.6
        overlay = np.dstack([vis] * 3)
        output = im.copy()
        cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
        gray = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)
    
    return gray

In [ ]:
from minisom import MiniSom

In [ ]:
def classify(som, data):
    """Classifies each sample in data in one of the classes definited
    using the method labels_map.
    Returns a list of the same length of data where the i-th element
    is the class assigned to data[i].
    """
    winmap = som.labels_map(X_train, y_train)
    default_class = np.sum(list(winmap.values())).most_common()[0][0]
    result = []
    for d in data:
        win_position = som.winner(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report





X_train, X_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.25)


som = MiniSom(300, 300, 300 , sigma=7, learning_rate=1, random_seed=10 , neighborhood_function='triangle')



 

#som.random_weights_init(imglst)
som.train_random(X_train, 25)


print(classification_report(y_test, classify(som, X_test)))

                       precision    recall  f1-score   support

Bacterial leaf blight       0.56      0.67      0.61        43
                blast       0.58      0.39      0.47        38
            brownspot       0.65      0.68      0.67        57

             accuracy                           0.60       138
            macro avg       0.59      0.58      0.58       138
         weighted avg       0.60      0.60      0.59       138

